In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import requests

In [ ]:
import seaborn as sns
sns.set_style("whitegrid")

%matplotlib inline

In [ ]:
url = 'https://pomber.github.io/covid19/timeseries.json'
with requests.get(url) as filedata:
    data = json.loads(filedata.content)

structure example:
> ```json
{
  "Afghanistan": [
    {
      "date": "2020-1-22",
      "confirmed": 0,
      "deaths": 0,
      "recovered": 0
    },
    ...
  ]
  ...
}
```

In [ ]:
country = 'Germany' # e.g., 'US', 'Italy', 'Philippines'
averages = 14

df = pd.json_normalize(data, record_path=country)
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

df['dt'] = np.gradient(df.get('confirmed'))
df['dt2'] = np.gradient(df.get('dt','confirmed'))
df['removed'] = df['recovered'] + df['deaths']
df['active'] = df['confirmed'] - df['recovered'] - df['deaths']

In [ ]:
sns.set(rc={'figure.figsize':(15,10)})
sns.set_style("whitegrid")
sns.lineplot(data=df, palette="tab10", linewidth=2.5, dashes=False);

In [ ]:
print("Total cases [{0}]:           \n{1}".format(country, df[-averages:]))
print("New cases rate [{0}]:        \n{1}".format(country, df['dt'][-averages:]))
print("New cases rate change [{0}]: \n{1}".format(country, df['dt2'][-averages:]))
print("Deaths [{0}]:                \n{1}".format(country, df['deaths'][-averages:]))
print("Removed cases [{0}]:         \n{1}".format(country, df['removed'][-averages:]))
print("Active cases [{0}]:          \n{1}".format(country, df['active'][-averages:]))

In [ ]:
intensive_care_capacity = 30000
intensive_care_factor = 1/100
infection_duration_days = 14

N_de = 100000000 # 1387297452 # 331002651 # 83019213
I_de = df['active'][-1]
R_de = df['removed'][-1]

beta_de = np.average(df['dt'][-averages:]) / np.average(df['active'][-averages:])
print("R (beta): {0}".format(beta_de))

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

# Total population, N.
N = N_de
# Initial number of infected and recovered individuals, I0 and R0.
I0, R0 = I_de, R_de
# Everyone else, S0, is susceptible to infection initially.
S0 = N - I0 - R0
# Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
beta, gamma = beta_de, 1./infection_duration_days
# A grid of time points (in days)
t = np.linspace(0, 500, 500)

# The SIR model differential equations.
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

# Initial conditions vector
y0 = S0, I0, R0
# Integrate the SIR equations over the time grid, t.
ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R = ret.T

# Plot the data on three separate curves for S(t), I(t) and R(t)
fig = plt.figure(facecolor='w')
ax = fig.add_subplot(111, axisbelow=True)
#ax.plot(t, S, 'b', alpha=0.5, lw=2, label='Susceptible')
ax.plot(t, I, 'r', alpha=0.5, lw=2, label='Active Cases')
ax.plot(t, R, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
ax.set_xlabel('Time /days')
ax.set_ylabel('Number')
#ax.set_ylim(0,1.2)
ax.yaxis.set_tick_params(length=0)
ax.xaxis.set_tick_params(length=0)
ax.grid(b=True, which='major', c='w', lw=2, ls='-')
legend = ax.legend()
legend.get_frame().set_alpha(0.5)
for spine in ('top', 'right', 'bottom', 'left'):
    ax.spines[spine].set_visible(False)
plt.show()

In [ ]:
df_pred = pd.DataFrame([df['active'].append(pd.Series(I)), 
                        df['removed'].append(pd.Series(R))])

transposed = df_pred.T
transposed.columns=['Active Cases', 'Removed']
transposed.reset_index()

transposed.plot(figsize=(15,10));

In [ ]:
sns.set(rc={'figure.figsize':(15,10)})
sns.set_style("whitegrid")
sns.lineplot(data=df['active'], palette="tab10", linewidth=2.5, dashes=False);
